In [1]:
import sys
sys.path.append('../')

from src.cmesrc.config import CMESRCV2_DB
import sqlite3
import pandas as pd

In [4]:
# conn = sqlite3.connect(CMESRCV2_DB)
# cur = conn.cursor()
# 
# # Want to add the widths and heights to HARPS table (columns already exist)
# cur.execute(
#     """
#     WITH SIZES AS (
#     SELECT HPBB.harpnum, MAX(HPBB.x_max - HPBB.x_min) as width, MAX(HPBB.y_max - HPBB.y_min) as height FROM HOURLY_PIXEL_BBOX HPBB
#     GROUP BY harpnum
#     )
#     UPDATE HARPS
#     SET pix_width = (SELECT width FROM SIZES WHERE SIZES.harpnum = HARPS.harpnum),
#         pix_height = (SELECT height FROM SIZES WHERE SIZES.harpnum = HARPS.harpnum)
#     WHERE HARPS.harpnum IN (SELECT harpnum FROM SIZES)
#     """
# )
# 
# conn.commit()
# conn.close()

In [3]:
conn = sqlite3.connect(CMESRCV2_DB)
conn.execute("PRAGMA foreign_keys = ON")
cur = conn.cursor()

In [9]:
query = """
SELECT S.*, FCHA.harpnum as mine, NHM.harpnum as theirs, FCHA.verification_score FROM SANJIV_SRC S
JOIN FINAL_CME_HARP_ASSOCIATIONS FCHA
ON FCHA.cme_id = S.cme_id
JOIN NOAA_HARPNUM_MAPPING NHM
ON NHM.noaa = S.noaa
"""

sanjiv = pd.read_sql(query, conn)
sanjiv["match"] = sanjiv["theirs"] == sanjiv["mine"]
sanjiv

,cme_id,cme_date,cme_width,cme_speed,flare_time,flare_class,noaa,mine,theirs,verification_score,match
0,20110427023605032,2011-04-27 02:36:00,257.0,924.0,2011-04-27 02:26:00,C2.0,11201,532,532,3.0,True
1,20110601183606112,2011-06-01 18:36:00,189.0,361.0,2011-06-01 16:51:00,C4.1,11226,637,637,4.0,True
2,20110602081206999,2011-06-02 08:12:00,360.0,976.0,2011-06-02 07:22:00,C3.7,11227,637,637,3.0,True
3,20110619164806120,2011-06-19 16:48:00,194.0,509.0,2011-06-19 16:03:00,C1.5,11237,674,674,4.0,True
4,20110621031610999,2011-06-21 03:16:00,360.0,719.0,2011-06-21 01:22:00,C7.7,11236,667,667,5.0,True
...,...,...,...,...,...,...,...,...,...,...,...
92,20140323033605999,2014-03-23 03:36:00,360.0,820.0,2014-03-23 02:27:00,C3.1,12014,3879,3879,3.0,True
93,20140328234807325,2014-03-28 23:48:00,138.0,514.0,2014-03-28 23:44:00,M2.6,12017,3894,3894,5.0,True
94,20140329024805032,2014-03-29 02:48:00,117.0,414.0,2014-03-29 01:54:00,C2.4,12024,3907,3907,3.0,True
95,20140329181205999,2014-03-29 18:12:00,360.0,528.0,2014-03-29 17:35:00,X1.0,12017,3894,3894,1.0,True


In [10]:
sanjiv[sanjiv.duplicated(subset="cme_id",keep=False)]

,cme_id,cme_date,cme_width,cme_speed,flare_time,flare_class,noaa,mine,theirs,verification_score,match
76,20130531121207118,2013-05-31 12:12:00,84.0,263.0,2013-05-31 11:11:00,C1.1,11761,2808,2790,3.0,False
77,20130531121207118,2013-05-31 12:12:00,84.0,263.0,2013-05-31 11:11:00,C1.1,11761,2808,2808,3.0,True


In [21]:
# So for cme_id 20130531121207118 noaa 11761 is associated with both harpnum 2790 and 2808. We chose 2808 so I will drop 2790 from the table.
sanjiv_clean = sanjiv[~ ((sanjiv["cme_id"] == 20130531121207118) & (sanjiv["theirs"] == 2790))]

In [23]:
# Describe the number of matches and missmatches by verification score

sanjiv_clean.groupby(["verification_score", "match"]).count()

,,cme_id,cme_date,cme_width,cme_speed,flare_time,flare_class,noaa,mine,theirs
verification_score,match,,,,,,,,,
1.0,True,36,36,36,36,36,36,36,36,36
2.0,True,14,14,14,14,14,14,14,14,14
3.0,True,19,19,19,19,19,19,19,19,19
4.0,True,15,15,15,15,15,15,15,15,15
5.0,True,12,12,12,12,12,12,12,12,12


In [26]:
majumdar = pd.read_sql("""
                       SELECT M.cme_id, M.matching_harps AS majumdar, FCHA.harpnum AS mine, FCHA.verification_score
                       FROM MAJUMDAR_SRC M
                       JOIN FINAL_CME_HARP_ASSOCIATIONS FCHA
                       ON M.cme_id = FCHA.cme_id
                       JOIN CMES C
                       ON C.cme_id = M.cme_id
                       WHERE (M.sr_type = "AR" OR M.sr_type = "AP") AND majumdar NOT NULL
                       """, conn)

majumdar["match"] = majumdar["majumdar"] == majumdar["mine"]

In [29]:
# Same for majumdar
majumdar.groupby(["verification_score", "match"])["cme_id"].count()

verification_score  match
1.0                 False     1
                    True     19
2.0                 False     5
                    True     24
3.0                 False     3
                    True     14
4.0                 False    12
                    True     50
5.0                 False    10
                    True     25
Name: cme_id, dtype: int64